In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CPU times: user 6.76 s, sys: 975 ms, total: 7.74 s
Wall time: 11.2 s


In [ ]:
import pandas as pd

# Define the corrected file paths
train_file_path = "/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/transliterated_task1.csv"
valid_file_path = "/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/validation_transliterated_task_1.csv"
test_file_path = "/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/test_transliterated.csv"

# Load the CSV files
train_data = pd.read_csv(train_file_path, delimiter=',', header=0)
valid_data = pd.read_csv(valid_file_path, delimiter=',', header=0)
test_data = pd.read_csv(test_file_path, delimiter=',', header=0)

In [ ]:
train_data = train_data[['transliterated_text', 'label']]

In [ ]:
train_data.head()

,transliterated_text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,മാഷ അല്ലാ,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. സി. അ. അ. യ്ക്ക് എതിരായ ...",Fake
3,ഇല്ലതെന്ത അവരു പുറത്ത് വിടയിട്ടല്ലണ്ട് വേറെന്ത്,Fake
4,ബറാന പക്ഷത്തിന് മറ്റൊരു നിയമം.നല്ല മൊയന്ത് മന്...,original


In [ ]:
train_data.shape

(3257, 2)

In [ ]:
train_data.columns

Index(['transliterated_text', 'label'], dtype='object')

In [ ]:
test_data.columns

Index(['Unnamed: 0', 'text', 'cleaned', 'transliterated_text'], dtype='object')

In [ ]:
# Assuming the correct column name is 'Unnamed: 0 ' (with trailing space)
train_data = train_data.drop(columns='Unnamed: 0', errors='ignore')

# Renaming columns
train_data = train_data.rename(columns={'text                ': 'text', 'category': 'label'})
# Stripping whitespaces from column names
valid_data.columns = valid_data.columns.str.strip()

# Dropping the column 'Unnamed: 0' (with possible leading/trailing spaces)
valid_data = valid_data.drop(columns='Unnamed: 0', errors='ignore')

# Renaming columns
valid_data = valid_data.rename(columns={'text                        ': 'text', 'category': 'label'})

test_data = test_data.rename(columns={'text                        ':'text'})

In [ ]:
train_data.head()

,transliterated_text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,മാഷ അല്ലാ,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. സി. അ. അ. യ്ക്ക് എതിരായ ...",Fake
3,ഇല്ലതെന്ത അവരു പുറത്ത് വിടയിട്ടല്ലണ്ട് വേറെന്ത്,Fake
4,ബറാന പക്ഷത്തിന് മറ്റൊരു നിയമം.നല്ല മൊയന്ത് മന്...,original


In [ ]:
# test_data = test_data[['transliterated_text']]
test_data.columns

Index(['Unnamed: 0', 'text', 'cleaned', 'transliterated_text'], dtype='object')

In [ ]:
train_data['label'].value_counts()

original    1658
Fake        1599
Name: label, dtype: int64

In [ ]:
train_data.shape

(3257, 2)

In [ ]:
valid_data['label'].value_counts()

original    409
Fake        406
Name: label, dtype: int64

In [ ]:
train_data['enc_label'] = train_data['label'].replace({'original':0, 'Fake':1
                                                       })
valid_data['enc_label'] = valid_data['label'].replace({'original':0, 'Fake':1
                                                       })

train_data.head()

,transliterated_text,label,enc_label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake,1
1,മാഷ അല്ലാ,Fake,1
2,"അന്വേഷണം കഴിയുമ്പോൾ,. സി. അ. അ. യ്ക്ക് എതിരായ ...",Fake,1
3,ഇല്ലതെന്ത അവരു പുറത്ത് വിടയിട്ടല്ലണ്ട് വേറെന്ത്,Fake,1
4,ബറാന പക്ഷത്തിന് മറ്റൊരു നിയമം.നല്ല മൊയന്ത് മന്...,original,0


In [ ]:
train_data['enc_label'].value_counts()

0    1658
1    1599
Name: enc_label, dtype: int64

In [ ]:
valid_data['enc_label'].value_counts()

0    409
1    406
Name: enc_label, dtype: int64

In [ ]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the validation data ==>", valid_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (3257, 3)
Size of the validation data ==> (815, 6)
Size of the Test data ==> (1019, 4)


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder=OneHotEncoder(sparse=False)
encoded_labels = pd.DataFrame (encoder.fit_transform(train_data[['label']]))
encoded_labels .columns = encoder.get_feature_names_out(['label'])
train_data= pd.concat([train_data, encoded_labels ], axis=1)

In [ ]:
train_data.columns

Index(['transliterated_text', 'label', 'enc_label', 'label_Fake',
       'label_original'],
      dtype='object')

In [ ]:
train_data = train_data.rename(columns={'label_original':'original','label_Fake':'Fake'
                                        })
train_data.sample(10)

,transliterated_text,label,enc_label,Fake,original
2829,എന്തുവാ സഖാവേ അമ്മായിക്കും അടുപ്പിലും ആവാം,original,0,0.0,1.0
134,"ഇവന്മാർ ഒരികളും സത്യം പറയില്ല, ലൈവ് റെക്കോർഡ് ...",Fake,1,1.0,0.0
2299,ഇതിലും വെല്യേ ഒരു ട്രോൾ ഉണ്ടോ അവതാരകന് എത്ര ക...,original,0,0.0,1.0
1610,പരനറികൾ.....,original,0,0.0,1.0
990,കഷ്ടം.....,original,0,0.0,1.0
1173,അപ്പോൾ ചിന ഉണ്ടാക്കി വിട്ട വൈറസ് അനോ ഇഇ കൊറോണ,Fake,1,1.0,0.0
1206,അഹ് പോയിട്ടോ,Fake,1,1.0,0.0
1869,ഇത് വെറും സാമ്പിള് വെടി കെട്ടു മാത്രം .,Fake,1,1.0,0.0
2986,പാകിസ്ഥാൻ അൽസോ ക്ലോസ് ഫ്രണ്ട്സ് ഗ്രൂപ്പ് അഫക്ട...,Fake,1,1.0,0.0
2668,സിപിഎം ന്റെ കൂടെ കൂടി വീണക്കും ബുന്ദി പോയി എന്...,original,0,0.0,1.0


In [ ]:
'''
Text Cleaning

'''
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  print(text)
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  # text = text.lower()
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  text = re.sub(r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+", ' ', text)
  print("---- DONE ----")
  return text


In [ ]:
punctuations = '''’'!()-[]{};:'"\,<>./?@#$%^&*_~�'''

def remove_punctuation_url(d):
    # d=d.lower()
    d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', d, flags=re.MULTILINE) #This line is for removing url
    review = d.replace('\n', '')
    no_punct = ""
    for char in review:
      if char not in punctuations:
         no_punct = no_punct + char
    return no_punct

def cleantext(string):
    text = string.split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)
    tx = text.replace('&amp',' ')
    #text = re.sub(r"[^0-9a-zA-Z]+",' ',text)
    # text = text.split()
    # text = [w for w in text if not w in stops]
    # text = " ".join(text)
    text = re.sub(r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+", ' ', text)

    return text

def clean(text):
  if not isinstance(text, str):
    return ''

  print('Text: ', text)
  text = remove_punctuation_url(text)
  text = cleantext(text)
  print('Output: ', text)
  print(" ")

  if not isinstance(text, str):
    return ''

  return text

In [ ]:
train_data.dropna(how='any', inplace=True)
valid_data.dropna(how='any', inplace=True)

In [ ]:
#removing URLs and Punctuations
#Removing punctuations
train_data['cleaned'] = train_data['transliterated_text'].apply(clean)
valid_data['cleaned'] = valid_data['transliterated_text'].apply(clean)
test_data['cleaned'] = test_data['transliterated_text'].apply(clean)


# ## Data samples after cleaning
# print("Data samples after cleaning:\n")
# for i in range(10):
#   print("Original Data:===\n",train_data.text[i],"\nCleaned Data:===\n",train_data.cleaned[i],)

In [ ]:
train_data.columns

Index(['transliterated_text', 'label', 'enc_label', 'Fake', 'original',
       'cleaned'],
      dtype='object')

In [ ]:
train_data.columns

Index(['transliterated_text', 'label', 'enc_label', 'Fake', 'original',
       'cleaned'],
      dtype='object')

In [ ]:
lables= [  'original','Fake']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    subset = dataset[dataset[l] == 1.0]
    word_list = [word.strip() for text in subset.cleaned for word in text.strip().split()]
    print(f"\nClass Name:--> {l} ")
    print("Number of Words:{}".format(len(word_list)))
    print("Number of Unique Words:{}".format(len(np.unique(word_list))))
    if word_list:
      print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
      print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    else:
      print("No posts in this class.")

In [ ]:
!pip install ktrain

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import ktrain
from ktrain import text

In [ ]:
categories = [ 'original','Fake']

In [ ]:
X_train = train_data['cleaned'].tolist()
y_train = train_data['enc_label'].tolist()

X_valid = valid_data['cleaned'].tolist()
y_valid = valid_data['enc_label'].tolist()

X_test = test_data['cleaned'].tolist()

In [ ]:
model_name = 'bert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=180,class_names=categories)

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: ml
train sequence lengths:
	mean : 11
	95percentile : 32
	99percentile : 67


Is Multi-Label? False
preprocessing test...
language: ml
test sequence lengths:
	mean : 10
	95percentile : 29
	99percentile : 59


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=16)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(2)}
weight

{0: 0.9827898550724637, 1: 1.0178236397748592}

In [ ]:
learner.fit_onecycle(1e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/10
204/204 [==============================] - 156s 676ms/step - loss: 0.6155 - accuracy: 0.6421 - val_loss: 0.6175 - val_accuracy: 0.7571
Epoch 2/10
204/204 [==============================] - 134s 659ms/step - loss: 0.4325 - accuracy: 0.8095 - val_loss: 0.4072 - val_accuracy: 0.8147
Epoch 3/10
204/204 [==============================] - 134s 659ms/step - loss: 0.3022 - accuracy: 0.8774 - val_loss: 0.3680 - val_accuracy: 0.8270
Epoch 4/10
204/204 [==============================] - 134s 659ms/step - loss: 0.2400 - accuracy: 0.9032 - val_loss: 0.3786 - val_accuracy: 0.8479
Epoch 5/10
204/204 [==============================] - 134s 659ms/step - loss: 0.1671 - accuracy: 0.9389 - val_loss: 0.4113 - val_accuracy: 0.8540
Epoch 6/10
 10/204 [>.............................] - ETA: 1:58 - loss: 0.0943 - accuracy: 0.9750

In [ ]:
learner.validate(class_names=categories)

In [ ]:
ktrain.get_predictor(learner.model,preproc=trans).save('m-bert-model')

In [ ]:
predictor = ktrain.load_predictor('m-bert-model')
model = ktrain.get_predictor(predictor.model, predictor.preproc)

In [ ]:
model_name = 'xlm-roberta-base'
trans = text.Transformer(model_name,maxlen=180,class_names=categories)

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

preprocessing train...
language: ml
train sequence lengths:
	mean : 11
	95percentile : 32
	99percentile : 67


Is Multi-Label? False
preprocessing test...
language: ml
test sequence lengths:
	mean : 10
	95percentile : 29
	99percentile : 59


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=16)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(2)}
weight

{0: 0.9827898550724637, 1: 1.0178236397748592}

In [ ]:
learner.fit_onecycle(1e-5,10,class_weight=weight)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/10
204/204 [==============================] - 169s 745ms/step - loss: 0.6764 - accuracy: 0.5677 - val_loss: 0.6059 - val_accuracy: 0.6871
Epoch 2/10
204/204 [==============================] - 151s 742ms/step - loss: 0.5356 - accuracy: 0.7475 - val_loss: 0.3974 - val_accuracy: 0.8245
Epoch 3/10
204/204 [==============================] - 151s 740ms/step - loss: 0.4305 - accuracy: 0.8074 - val_loss: 0.3721 - val_accuracy: 0.8466
Epoch 4/10
204/204 [==============================] - 151s 741ms/step - loss: 0.3672 - accuracy: 0.8458 - val_loss: 0.3489 - val_accuracy: 0.8454
Epoch 5/10
204/204 [==============================] - 151s 741ms/step - loss: 0.3056 - accuracy: 0.8753 - val_loss: 0.3414 - val_accuracy: 0.8613
Epoch 6/10
204/204 [==============================] - 151s 741ms/step - loss: 0.2611 - accuracy: 0.8998 - val_loss: 0.3483 - val_accuracy: 0.8650
Epoch 7/10
204/204 [==============================] - 151s 74

In [ ]:
learner.validate(class_names=categories)

26/26 [==============================] - 13s 391ms/step
              precision    recall  f1-score   support

    original       0.88      0.88      0.88       409
        Fake       0.88      0.88      0.88       406

    accuracy                           0.88       815
   macro avg       0.88      0.88      0.88       815
weighted avg       0.88      0.88      0.88       815



array([[360,  49],
       [ 50, 356]])

In [ ]:
ktrain.get_predictor(learner.model,preproc=trans).save('/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/roberta-model')

In [ ]:
predictor = ktrain.load_predictor('/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/roberta-model')
model = ktrain.get_predictor(predictor.model, predictor.preproc)

In [ ]:
model_name = 'l3cube-pune/malayalam-bert'
trans = text.Transformer(model_name,maxlen=180,class_names=categories)

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)

In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=9)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes = np.unique(train_data['enc_label'])
                                               ,y = train_data['enc_label'])
class_weights = dict(zip(np.unique(train_data['enc_label']), class_weights))

In [ ]:
weight = {i : class_weights[i] for i in range(2)}
weight

In [ ]:
learner.fit_onecycle(1e-5,10,class_weight=weight)

In [ ]:
learner.validate(class_names=categories)

In [ ]:
ktrain.get_predictor(learner.model,preproc=trans).save('malay-model')

In [ ]:
predictor = ktrain.load_predictor('/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/roberta-model')
model = ktrain.get_predictor(predictor.model, predictor.preproc)

In [ ]:
predictor.predict('എന്തുവാ സഖാവേ അമ്മായിക്കും അടുപ്പിലും ആവാം	')

'original'

In [ ]:
test_data.sample(10)

,Unnamed: 0,text,cleaned,transliterated_text,Label
234,234,China cheated the whole world. World should bo...,ചിന ചീറ്റഡ് തെ വോൾ വേർൽഡ് വേർൽഡ് ഷോൾഡ് ബോയ്കോട...,ചിന ചീറ്റഡ് തെ വോൾ വേർൽഡ് വേർൽഡ് ഷോൾഡ് ബോയ്കോട...,Fake
742,742,Kashtam,കഷ്ടം,കഷ്ടം,original
833,833,ശരി യാകാൻ ഇടയില്ല. <br>.. ചൈനയിൽ ഇപ്പോഴും മരണം...,ശരി യാകാൻ ഇടയില്ല ബിആർ ചൈനയിൽ ഇപ്പോഴും മരണം കൊ...,ശരി യാകാൻ ഇടയില്ല ബിആർ ചൈനയിൽ ഇപ്പോഴും മരണം കൊ...,Fake
361,361,ദൈവം മനുഷ്യനെ സരള ഹൃദയനായി സൃഷ്ടിച്ചു. എന്നാല്...,ദൈവം മനുഷ്യനെ സരള ഹൃദയനായി സൃഷ്ടിച്ചു എന്നാല്‍...,ദൈവം മനുഷ്യനെ സരള ഹൃദയനായി സൃഷ്ടിച്ചു എന്നാല്‍...,Fake
5,5,Why not all countries club together n block ch...,വൈ നോട്ട് ആൽ കണ്ട്രീസ് ക്ലബ് ടോഗെതർ ന് ബ്ലോക്ക...,വൈ നോട്ട് ആൽ കണ്ട്രീസ് ക്ലബ് ടോഗെതർ ന് ബ്ലോക്ക...,Fake
832,832,എന്തായാലും മരണം നിശ്ചയം. അതറിയാവുന്നവർ ആയിരിക്...,എന്തായാലും മരണം നിശ്ചയം അതറിയാവുന്നവർ ആയിരിക്ക...,എന്തായാലും മരണം നിശ്ചയം അതറിയാവുന്നവർ ആയിരിക്ക...,Fake
219,219,Nashicha vaccine edukandayirunnu kop 😡,നശിച്ച വാക്സിൻ എടുകണ്ടയിരുന്നു കോപ്പ്,നശിച്ച വാക്സിൻ എടുകണ്ടയിരുന്നു കോപ്പ് 😡,Fake
960,960,Sagav p jayarjettanu ethire nadapadi ethinayir...,സാഗവ് പ് ജയർജെറ്റാണ് എതിരെ നടപടി ഇതിനായിരുന്നു...,സാഗവ് പ് ജയർജെറ്റാണ് എതിരെ നടപടി ഇതിനായിരുന്നു...,original
654,654,ഇതേക്കുറിച്ചു ഞാനും ചിന്തിച്ചിരുന്നു....... എന...,ഇതേക്കുറിച്ചു ഞാനും ചിന്തിച്ചിരുന്നു എന്തായാലു...,ഇതേക്കുറിച്ചു ഞാനും ചിന്തിച്ചിരുന്നു എന്തായാലു...,Fake
867,867,vidagdar ethiyallo 🤣🤣🤣🤣🤣🤣🤣🤣,വിദഗ്ദർ എത്തിയല്ലോ,വിദഗ്ദർ എത്തിയല്ലോ 🤣🤣🤣🤣🤣🤣🤣🤣,Fake


In [ ]:
test_data['Label'] = test_data['cleaned'].apply(predictor.predict)

In [ ]:
test_data.sample(10)

,Unnamed: 0,text,cleaned,transliterated_text,Label
506,506,തിരുവാതിര കളിക്കാന്‍ Parda ക്കാരെ ആരെയും കണ്ടി...,തിരുവാതിര കളിക്കാന്‍ പർദ ക്കാരെ ആരെയും കണ്ടില്...,തിരുവാതിര കളിക്കാന്‍ പർദ ക്കാരെ ആരെയും കണ്ടില്...,original
57,57,ഇതു സത്യമണ്ണെകിൽ എല്ലാ രാഷ്ട്രങ്ങൾ ഒത്തു ചേർന്...,ഇതു സത്യമണ്ണെകിൽ എല്ലാ രാഷ്ട്രങ്ങൾ ഒത്തു ചേർന്...,ഇതു സത്യമണ്ണെകിൽ എല്ലാ രാഷ്ട്രങ്ങൾ ഒത്തു ചേർന്...,Fake
167,167,എന്നിട്ടും പ്രവാസികൾക്ക് ആണ് നിർബന്ധിത ക്വാറന്...,എന്നിട്ടും പ്രവാസികൾക്ക് ആണ് നിർബന്ധിത ക്വാറന്...,എന്നിട്ടും പ്രവാസികൾക്ക് ആണ് നിർബന്ധിത ക്വാറന്...,original
304,304,അധികാരത്തിൽ എത്തിക്കഴിയുമ്പോൾ എല്ലാവരും ഇതേരീത...,അധികാരത്തിൽ എത്തിക്കഴിയുമ്പോൾ എല്ലാവരും ഇതേരീത...,അധികാരത്തിൽ എത്തിക്കഴിയുമ്പോൾ എല്ലാവരും ഇതേരീത...,original
760,760,Nalla role model government,നല്ല റോൾ മോഡൽ ഗവർണ്മെന്റ്,നല്ല റോൾ മോഡൽ ഗവർണ്മെന്റ്,original
375,375,Cpm thanne ee parupadi kal ellam cheyunnathe j...,സിപിഎം തന്നെ ഇഇ പരുപാടി കാൽ എല്ലാം ചെയുന്നതെ ജ...,സിപിഎം തന്നെ ഇഇ പരുപാടി കാൽ എല്ലാം ചെയുന്നതെ ജ...,original
64,64,Panna rascal paranaari...,പന്ന റാസ്കൽ പരനാറി,പന്ന റാസ്കൽ പരനാറി,Fake
15,15,പല വസ്തുതകൾ പഠന വിധേയമാക്കിയിട്ടില്ല . ചൈന ...,പല വസ്തുതകൾ പഠന വിധേയമാക്കിയിട്ടില്ല ചൈന മാത്ര...,പല വസ്തുതകൾ പഠന വിധേയമാക്കിയിട്ടില്ല ചൈന മ...,Fake
916,916,എന്ന പിന്നെ കൊറോണ കഴിഞ ഒരു യുദ്ധവും പ്രതീക്ഷിക...,എന്ന പിന്നെ കൊറോണ കഴിഞ ഒരു യുദ്ധവും പ്രതീക്ഷിക...,എന്ന പിന്നെ കൊറോണ കഴിഞ ഒരു യുദ്ധവും പ്രതീക്ഷിക...,Fake
397,397,പിണറായി ഭഗവാനും വീണ ഭഗവതിയും നമ്മളെ രക്ഷിക്കും🙏😥🙏,പിണറായി ഭഗവാനും വീണ ഭഗവതിയും നമ്മളെ രക്ഷിക്കും,പിണറായി ഭഗവാനും വീണ ഭഗവതിയും നമ്മളെ രക്ഷിക്കും🙏😥🙏,original


In [ ]:
final_output = test_data[['text', 'Label']]
final_output = final_output.rename(columns={'text': 'Text'})

final_output.sample(10)

,Text,Label
477,Pravasikal ano koronakkar...,original
554,ഇ രാഷ്ട്രീയക്കാർക്ക് എന്നാ നേരം വെളുക്കുക വീണ്...,original
622,എന്റെ പാറശാലയെ കാത്തോളണേ 😂😂😂,original
550,ividathepole azhinjadan samathikkilla thattik...,Fake
304,അധികാരത്തിൽ എത്തിക്കഴിയുമ്പോൾ എല്ലാവരും ഇതേരീത...,original
888,Ini aryippu Varum veettil vare samoohika akala...,original
72,കൊറോണ സിപിഎം നേയും dyfi. യേയും ഭയക്കുന്നു. ബക്...,original
208,ഇവന്മാരെ വീണ്ടും ജയിപ്പിച്ചു വിട്ടത് ആണ് നമ്മൾ...,original
451,Maha kallan,original
488,"Ethinokke paranju vidunna, veetukkare paranjal...",original


In [ ]:
final_output.to_csv('/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/Punny_Punctuators_task1_run2.csv') # roberta

In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/Punny_Punctuators_task1_run1.csv')
dataframe = dataframe[['Text', 'Label']]
dataframe.sample(10)

,Text,Label
291,Comedy show ayi poyi thiruvathira,original
837,സിപിഎം സമ്മേളനങ്ങൾ എല്ലാം കഴിഞ്ഞാൽ കേരളം ലോക്ക...,original
338,WHO ക്ക് ഫണ്ട്‌ ചെയ്യുന്ന ചൈന 😑,Fake
432,ഗംഗ പരിശുദ്ധമാണ് കോവിഡ് രോഗിയും ഗംഗയിൽ കുളിച്ച...,Fake
479,ചൈന ഒരു അത്ഭുദ മാണ്,Fake
1007,ലോകം മുഴുവനും ശോഭിച് ഇല്ലെങ്കിലും ഇന്ത്യയിൽ ഒന...,Fake
501,Corona chinayude jaiva ayudhamaanenne nerathe ...,Fake
739,ഭയം വേണ്ട ജാഗ്രത മതി😂,original
633,Evarokke varum എല്ലാം Sheri aakum!! Nalla bhar...,original
329,നാണമില്ലേ ഇവർക്ക് പറയുന്നത് ഒന്ന് ചെയുന്നത് മ...,original


In [ ]:
dataframe.to_csv('/content/drive/MyDrive/Punny_Punctuators/DravidianLangTech@EACL_2024/Task_1/Punny_Punctuators_task1_run3.csv', index=False)